In [1]:
import json
from typing import Sequence

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_hub.semanticscholar.base import SemanticScholarReader

/Users/shaurya/projects/S2QA/.conda/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing BasePromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/Users/shaurya/projects/S2QA/.conda/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(


In [2]:
# try:
#     storage_context = StorageContext.from_defaults(persist_dir="./storage/biases")
#     march_index = load_index_from_storage(storage_context)

#     storage_context = StorageContext.from_defaults(persist_dir="./storage/methods")
#     june_index = load_index_from_storage(storage_context)

#     storage_context = StorageContext.from_defaults(persist_dir="./storage/datasets")
#     sept_index = load_index_from_storage(storage_context)

#     index_loaded = False
# except:
#     index_loaded = False
    
index_loaded = False

In [3]:
# build indexes across the three data sources
s2reader = SemanticScholarReader()

if not index_loaded:
    # load data
    biases_docs = s2reader.load_data(
        query="anti muslim biases in large language models", limit=10
    )
    methods_docs = s2reader.load_data(
        query="methods to reduce biases in large language models", limit=10
    )
    datasets_docs = s2reader.load_data(
        query="datasets to reduce biases in large language models", limit=10
    )
    mitigation_strategies_docs = s2reader.load_data(
        query="mitigation strategies for biases in large language models", limit=10
    )
    evaluation_metrics_docs = s2reader.load_data(
        query="evaluation metrics for bias detection in large language models", limit=10
    )
    research_challenges_docs = s2reader.load_data(
        query="research challenges in bias reduction for large language models",
        limit=10,
    )
    case_studies_docs = s2reader.load_data(
        query="case studies of bias reduction in large language models", limit=10
    )

    industry_practices_docs = s2reader.load_data(
        query="industry practices in bias reduction for large language models", limit=10
    )
    # build index
    biases_index = VectorStoreIndex.from_documents(biases_docs)
    methods_index = VectorStoreIndex.from_documents(methods_docs)
    datasets_index = VectorStoreIndex.from_documents(datasets_docs)
    mitigation_strategies_index = VectorStoreIndex.from_documents(
        mitigation_strategies_docs
    )
    evaluation_metrics_index = VectorStoreIndex.from_documents(evaluation_metrics_docs)
    research_challenges_index = VectorStoreIndex.from_documents(
        research_challenges_docs
    )
    case_studies_index = VectorStoreIndex.from_documents(case_studies_docs)
    industry_practices_index = VectorStoreIndex.from_documents(industry_practices_docs)

    # persist index
    biases_index.storage_context.persist(persist_dir="./storage/biases")
    methods_index.storage_context.persist(persist_dir="./storage/methods")
    datasets_index.storage_context.persist(persist_dir="./storage/datasets")
    mitigation_strategies_index.storage_context.persist(
        persist_dir="./storage/mitigation_strategies"
    )
    evaluation_metrics_index.storage_context.persist(
        persist_dir="./storage/evaluation_metrics"
    )
    research_challenges_index.storage_context.persist(
        persist_dir="./storage/research_challenges"
    )
    case_studies_index.storage_context.persist(persist_dir="./storage/case_studies")
    industry_practices_index.storage_context.persist(
        persist_dir="./storage/industry_practices"
    )

In [4]:
biases_engine = biases_index.as_query_engine(similarity_top_k=3)
methods_engine = methods_index.as_query_engine(similarity_top_k=3)
datasets_engine = datasets_index.as_query_engine(similarity_top_k=3)
mitigation_strategies_engine = mitigation_strategies_index.as_query_engine(
    similarity_top_k=3
)
evaluation_metrics_engine = evaluation_metrics_index.as_query_engine(similarity_top_k=3)
research_challenges_engine = research_challenges_index.as_query_engine(
    similarity_top_k=3
)
case_studies_engine = case_studies_index.as_query_engine(similarity_top_k=3)
industry_practices_engine = industry_practices_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=biases_engine,
        metadata=ToolMetadata(
            name="llm_biases",
            description="This has information about anti muslim biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=methods_engine,
        metadata=ToolMetadata(
            name="llm_bias_methods",
            description="This has information about methods to reduce biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=datasets_engine,
        metadata=ToolMetadata(
            name="llm_bias_datasets",
            description="This has information about datasets to reduce biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=mitigation_strategies_engine,
        metadata=ToolMetadata(
            name="llm_bias_mitigation_strategies",
            description="This has information about mitigation strategies to reduce biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=evaluation_metrics_engine,
        metadata=ToolMetadata(
            name="llm_bias_evaluation_metrics",
            description="This has information about evaluation metrics to measure biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=research_challenges_engine,
        metadata=ToolMetadata(
            name="llm_bias_research_challenges",
            description="This tool contains information about the challenges faced in the field of bias research in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=case_studies_engine,
        metadata=ToolMetadata(
            name="llm_bias_case_studies",
            description="This tool has information about case studies on biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=industry_practices_engine,
        metadata=ToolMetadata(
            name="llm_bias_industry_practices",
            description="This tool provides information on industry practices for mitigating biases in large language models. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
]

In [6]:
from llama_index.schema import Document
from llama_index.agent import ContextRetrieverOpenAIAgent


# toy index - stores a list of abbreviations
texts = [""]
docs = [Document(text=t) for t in texts]
context_index = VectorStoreIndex.from_documents(docs)

context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    query_engine_tools, context_index.as_retriever(similarity_top_k=1), verbose=True
)

In [11]:
response = context_agent.chat(
    "Think of a new  research techniques on bias reduction in large language models."
)

Context information is below.
---------------------

---------------------
Given the context information and not prior knowledge, either pick the corresponding tool or answer the function: Think of a new  research techniques on bias reduction in large language models.

=== Calling Function ===
Calling function: llm_bias_research_challenges with args: {
"input": "research techniques on bias reduction in large language models"
}
Got output: The paper "Adversarial Filters of Dataset Biases" presented at the International Conference on Machine Learning in 2020 investigates the use of adversarial filters to mitigate the overestimation of machine performance caused by dataset biases in large language models. The authors provide a theoretical understanding of the approach and demonstrate its effectiveness in reducing measurable dataset biases. They also show that models trained on filtered datasets exhibit better generalization to out-of-distribution tasks. The paper highlights the research c

In [12]:
from IPython.display import Markdown

# print markdown
Markdown(str(response))

One research technique on bias reduction in large language models is the use of adversarial filters to mitigate dataset biases. This technique involves identifying and filtering out biased examples from the training data to reduce the overestimation of machine performance caused by these biases.

The research paper "Adversarial Filters of Dataset Biases" presented at the International Conference on Machine Learning in 2020 explores this approach. The authors provide a theoretical understanding of adversarial filters and demonstrate their effectiveness in reducing measurable dataset biases. They also show that models trained on filtered datasets exhibit better generalization to out-of-distribution tasks.

This research technique addresses the challenge of dataset biases and their impact on model performance. By filtering out biased examples, the models can be trained to be more robust and generalize better to diverse tasks and domains.

It is important to note that this is just one research technique among many others being explored in the field of bias reduction in large language models. Ongoing research aims to develop new and innovative techniques to further mitigate biases and improve the fairness and inclusivity of these models.

# Raw GPT-4 Output

In order to reduce anti-Muslim biases or any other type of biases in large language models, a variety of strategies can be applied:

1. Diverse Training Data: The input data that is used to train algorithms should be diverse and include various types of linguistic input from different regions, communities, religions, and societies. If a model is trained on biased or unrepresentative data, it will produce biased results.

2. Moderation and Regulation: Content moderation can be used to filter out any discriminatory, biased, or harmful content that may affect the learning of the model. This could include setting stringent rules for data collection, processing, and application.

3. Use of Advanced Technologies: Machine learning and AI can be used to identify and eliminate any bias in the algorithm or data. Companies can leverage technologies like AI to detect prejudicial patterns and remove them.

4. Bias Evaluation: Frequent evaluations and audits can help determine if any bias has seeped into the system. This will help maintain the neutrality and fairness of content.

5. Raising Awareness: It is important to educate the AI community about the harm of biases and the need to actively avoid them during development.

6. Strict Policies: Implementing strict anti-discrimination policies and practices within tech companies can help prevent biases from creeping into AI systems.

7. Continuous learning and retraining: Language models should be trained on an ongoing basis with updated and diverse data to avoid any sort of biases.

8. Inclusion of Ethics in AI: Including aspects of ethical considerations and principles in AI.

The solution is a combination of developing robust technical approaches and fostering an inclusive culture in AI research and development.
